# Fake News Classifier Using LSTM

# Importing Libraries and dataset:

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('E:/Data Science/Data Sets/Fake News/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Simple preprocessing:

In [3]:
###Drop Nan Values
df=df.dropna()

In [4]:
## Get the Independent Features and Dependent features
X=df.drop('label',axis=1)
y=df['label']

In [5]:
X.shape, y.shape

((18285, 4), (18285,))

## Importing Tensorflow libraries

In [6]:
import tensorflow as tf

In [7]:
tf.__version__

'2.4.1'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# Text Preprocessing:

In [9]:
### Vocabulary size
voc_size=5000

In [10]:
messages=X.copy()

In [11]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipul\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

### Dataset Preprocessing:

In [15]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [16]:
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [18]:
corpus[0:4]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi']

## One Hot Representation:

In [20]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0:5]

[[1648, 1578, 2415, 516, 2695, 1572, 1756, 4803, 4978, 4252],
 [2359, 565, 3666, 971, 2911, 2027, 4836],
 [3185, 4415, 1060, 3415],
 [4387, 324, 4460, 2922, 138, 2547],
 [3091, 2911, 2508, 1554, 2558, 4568, 2911, 3886, 3270, 3794]]

### Embedding Representation

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4803 4978 4252]
 [   0    0    0 ... 2911 2027 4836]
 [   0    0    0 ... 4415 1060 3415]
 ...
 [   0    0    0 ...  312 1309 3371]
 [   0    0    0 ... 2814  303 3744]
 [   0    0    0 ... 2768 4321 2680]]


In [22]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1648,
       1578, 2415,  516, 2695, 1572, 1756, 4803, 4978, 4252])

# Creating model:

In [23]:
embedding_vector_features=40

In [24]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

In [25]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


# Preparing the train and test dataset:

In [26]:
len(embedded_docs),y.shape

(18285, (18285,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Training The Model:

In [30]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Epoch 1/5
192/192 [==============================] - 29s 89ms/step - loss: 0.4883 - accuracy: 0.7312 - val_loss: 0.2118 - val_accuracy: 0.9158
Epoch 2/5
192/192 [==============================] - 8s 43ms/step - loss: 0.1483 - accuracy: 0.9451 - val_loss: 0.2087 - val_accuracy: 0.9210
Epoch 3/5
192/192 [==============================] - 6s 33ms/step - loss: 0.1009 - accuracy: 0.9635 - val_loss: 0.2018 - val_accuracy: 0.9157
Epoch 4/5
192/192 [==============================] - 6s 33ms/step - loss: 0.0839 - accuracy: 0.9698 - val_loss: 0.2365 - val_accuracy: 0.9180
Epoch 5/5
192/192 [==============================] - 6s 33ms/step - loss: 0.0542 - accuracy: 0.9826 - val_loss: 0.2759 - val_accuracy: 0.9190


## Performance Metrics And Accuracy

In [31]:
y_pred = model.predict_classes(X_test)

C:\Users\vipul\anaconda3\envs\vipy3-TF2.0\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[3048,  371],
       [ 118, 2498]], dtype=int64)

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9189726594863298